In [7]:
from utility import *
import operator
from itertools import product
from itertools import accumulate
import numpy as np
import random

import pickle

from scipy.interpolate import BSpline

from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from numpy.linalg import inv
from functools import reduce

from scipy.stats import norm
from scipy import integrate

from model.AGENT import *


import gym
import itertools
import numpy as np
import sys
# if "../" not in sys.path:
#   sys.path.append("../") 
from collections import defaultdict
from lib.envs.cliff_walking import CliffWalkingEnv
from lib.envs.windy_gridworld import WindyGridworldEnv

from scipy.optimize import minimize, rosen, rosen_der
from scipy.optimize import Bounds
from tqdm import tqdm





env = CliffWalkingEnv()
Q_space = np.load("Q-table-cliff.npz")["xxx"]
Q_space2 = np.load("Q-table-cliff.npz")["xxx"]

prob1 = [1.0 for i in range((env.nA))]
prob1 = prob1/np.sum(prob1)

###################
## discount rate ##
###################
gamma = 0.7
###################################
## parameter for behavior policy ## larger --> more expert 
###################################
betabeta = 0.5
#################################
## parameter for target policy ##
#################################
alpha = 1
#################################
## parameter for MC repetition ##
#################################
n_MC = 20000
sample_size = 1000

def sample_policy(observation, alpha = alpha):
    prob2 = alpha*Q_space[observation,:] +(1-alpha)*prob1
    return np.random.choice(env.nA,1,p=prob2)[0] ## 4个action 选一
        
def behavior_policy(observation,beta=betabeta):
    prob2 = beta*Q_space[observation,:]+ (1-beta)*prob1 
    return np.random.choice(env.nA,1,p=prob2)[0]

a = simulation(env, n = sample_size, reward_dicount = gamma, scale = "Cliffwalk")

output, _, _ = a.evaluate_policy(policy = sample_policy, seed = None, S_init = None, n = n_MC) 
est_mean = np.mean(output)
est_mean, np.std(output)

100%|██████████| 20000/20000 [00:31<00:00, 631.43it/s]


(-3.2946097756811317, 0.7016945263508425)

In [1]:
## check infer
sample_size = 500

env = CliffWalkingEnv()


Q_space = np.load("Q-table-cliff.npz")["xxx"]
Q_space2 = np.load("Q-table-cliff.npz")["xxx"]

prob1 = [1.0 for i in range((env.nA))]
prob1 = prob1/np.sum(prob1)


betabeta = 0.5
alpha = 1
def sample_policy(observation,alpha= alpha):
    prob2 = alpha*Q_space[observation,:] +(1-alpha)*prob1
    return np.random.choice(env.nA,1,p=prob2)[0] ## 4个action 选一
        
def behavior_policy(observation,beta=betabeta):
    prob2 = beta*Q_space[observation,:]+ (1-beta)*prob1 ## behavior这么聪明？
    return np.random.choice(env.nA,1,p=prob2)[0]

a = simulation(env, n = sample_size, reward_dicount = gamma, scale = "Cliffwalk_noise")


rep = 100
filename_CI = 'CI_store_cliff_walk_fixed'
outfile_CI = open(filename_CI, 'ab')
est_mean = est_mean
count = 0
AL = []
pred_all_basis = []
for i in range(rep):
    try:
        a.buffer = {} ## when using gen_buffer, we should empty the buffer first!!
        a.gen_buffer(total_N = None, S_init = None, policy = behavior_policy )
        a.B_spline_degrade()
        lower_bound, upper_bound = a.inference(policy = sample_policy, S = 36)
        print("obtain bounds:", lower_bound, upper_bound, "estimated value", (lower_bound + upper_bound)/2 )
        pickle.dump([lower_bound,upper_bound], outfile_CI)
        AL.append(upper_bound - lower_bound)
        if lower_bound < est_mean and est_mean < upper_bound:
            count += 1
        pred_all_basis.append((lower_bound + upper_bound) / 2)
        #if i % 10 == 0 :
        #    print("iteration,", i , count, 
        #      "CI", lower_bound, upper_bound, "sigma_2", a.sigma2, 
        #      "estimated mean", np.mean([lower_bound[0], upper_bound[0]]))

        print(count / len(AL), np.mean(AL), np.mean(pred_all_basis) )
    except:
        print("+" * 10 +" 1 " + "+" * 10)
outfile_CI.close()



In [9]:
print("for sample size", sample_size, "gamma" , gamma )
def mse(aaa, true = -3.301610434961245):
    aaa = np.array(aaa)
    aaa = aaa[aaa>-100]
    return [np.mean((((aaa-true)*(aaa-true)))),np.sqrt(np.var((aaa-true)*(aaa-true)))]
print("ourmethod: CI %.3f, AL %.3f, pred %.3f, MSE %.5f(%.5f)" %(count / len(AL),  np.mean(AL), np.mean(pred_all_basis), *mse(pred_all_basis, est_mean)))

for sample size 500 gamma 0.7
ourmethod: CI 0.967, AL 0.143, pred -3.299, MSE 0.00113(0.00140)
